[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ninjaneural/sagemaker/blob/main/notebook/webui_xl.ipynb)

In [3]:
# 실행변수
HomePath = '/home/studio-lab-user'
Workspace = 'ui'


In [ ]:
# 설치
HomePath = '/home/studio-lab-user'

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/121931?type=Model&format=SafeTensor&size=full&fp=fp16' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = 'DreamShaper_XL_09Alpha.safetensors' #@param {type:"string"}

#**ControlNet을 사용하지 않으면 체크를 해제해주세요**
ControlNet = False

#**After Detailer(adetailer)를 사용하지 않으면 체크를 해제해주세요**
ADetailer = False

#**Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = False

#**Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False

Workspace = 'ui'
TagVersion = 'v1.6.0'

!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q xformers==0.0.20 triton==2.0.0 -U

# install util
!conda install -q -y aria2
!conda install glib=2.51.0 -y

%cd {HomePath}
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/install_sagemaker.sh -O ./install.sh
!bash install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything}

%cd {HomePath}/{Workspace}
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/config_api.json -O ./config.json

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' ./config.json

# 추가 익스텐션
# git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-haku-img ./extensions/a1111-sd-webui-haku-img

# checkpoint 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d ./models/Stable-diffusion -o {Checkpoint_Filename}

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d ./models/ESRGAN -o 4x-UltraSharp.pth

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d ./models/Stable-diffusion -o vae-ft-mse-840000-ema-pruned.vae.pt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/87153?type=Model&format=SafeTensor" -d ./models/Lora -o more_details.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d ./embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d ./embeddings -o badhandv4.pt

!conda clean --all -y
!rm -rf {HomePath}/.cache

print("설치 완료! 시작을 실행해주세요")


In [ ]:
# controlnet model 다운로드
if 'HomePath' in locals():
    %cd {HomePath}/{Workspace}

    # 모델삭제
    !rm ./extensions/controlnet/models/*

    # # controlnet model
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors -d ./extensions/controlnet/models -o diffusers_xl_canny_mid.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors -d ./extensions/controlnet/models -o diffusers_xl_depth_mid.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors -d ./extensions/controlnet/models -o sargezt_xl_softedge.safetensors
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors -d ./extensions/controlnet/models -o thibaud_xl_openpose.safetensors

#     # anime (kohya)
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors -d ./extensions/controlnet/models -o kohya_controllllite_xl_canny_anime.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors -d ./extensions/controlnet/models -o kohya_controllllite_xl_depth_anime.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors -d ./extensions/controlnet/models -o kohya_controllllite_xl_openpose_anime_v2.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors -d ./extensions/controlnet/models -o kohya_controllllite_xl_scribble_anime.safetensors

#     # sai
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_256lora.safetensors -d ./extensions/controlnet/models -o sai_xl_canny_256lora.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_256lora.safetensors -d ./extensions/controlnet/models -o sai_xl_depth_256lora.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors -d ./extensions/controlnet/models -o sai_xl_recolor_256lora.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors -d ./extensions/controlnet/models -o sai_xl_sketch_256lora.safetensors

#     # t2i adapter
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors -d ./extensions/controlnet/models -o t2i-adapter_xl_canny.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors -d ./extensions/controlnet/models -o t2i-adapter_xl_openpose.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors -d ./extensions/controlnet/models -o t2i-adapter_xl_sketch.safetensors

#     #etc
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth -d ./extensions/controlnet/models -o ip-adapter_xl.pth

#     # temporalnet
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/controlnet-temporalnet-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors -d /content/$1/extensions/controlnet/models -o diffusion_pytorch_model.safetensors


In [4]:
# 다른 모델(checkpoint) 다운로드
Checkpoint_Url = 'https://civitai.com/api/download/models/121931?type=Model&format=SafeTensor&size=full&fp=fp16' #@param {type:"string"}
Checkpoint_Filename = 'DreamShaper_XL_09Alpha.safetensors' #@param {type:"string"}

if 'HomePath' in locals():
    %cd {HomePath}/{Workspace}

    # 모델삭제
    !rm ./models/Stable-diffusion/*

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d ./models/Stable-diffusion -o {Checkpoint_Filename}


/home/studio-lab-user/ui
[#04278e 6.4GiB/6.4GiB(99%) CN:11 DL:92MiB]0m]mm
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
04278e|OK  |   122MiB/s|./models/Stable-diffusion/DreamShaper_XL_09Alpha.safetensors

Status Legend:
(OK):download completed.


In [ ]:
# 시작 (localtunnel 터널링)

%cd {HomePath}/{Workspace}
!python launch.py --xformers --no-half-vae --enable-insecure-extension-access --theme dark --ngrok 2O12b7KodfUC93I9kGlOxkbyU1i_4m4dWcVktVGBfBDZAFnqC

In [ ]:
# 저장

!tar czvf {HomePath}/output.tar.gz {HomePath}/{Workspace}/outputs
!rm -r {HomePath}/{Workspace}/outputs


In [ ]:
# 삭제

!rm {HomePath}/output.tar.gz

In [6]:
# 용량 확인
!df -h {HomePath}

!du -sh {HomePath}/.[^.]*
!du -sh {HomePath}/*

Cache location: 
There are no tarballs to remove
Cache location: 
There are no unused packages to remove
Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme1n1     25G   24G  1.8G  94% /home/studio-lab-user
4.0K	/home/studio-lab-user/.bashrc
12G	/home/studio-lab-user/.conda
0	/home/studio-lab-user/.config
36K	/home/studio-lab-user/.ipynb_checkpoints
584K	/home/studio-lab-user/.ipython
24K	/home/studio-lab-user/.jupyter
4.0K	/home/studio-lab-user/.keras
124K	/home/studio-lab-user/.local
4.0K	/home/studio-lab-user/.ngrok2
3.8M	/home/studio-lab-user/.npm
0	/home/studio-lab-user/.nv
0	/home/studio-lab-user/.sagemaker
4.0K	/home/studio-lab-user/.wget-hsts
192K	/home/studio-lab-user/sagemaker-studiolab-notebooks
12G	/home/studio-lab-user/ui
20K	/home/studio-lab-user/webui.ipynb
16K	/home/studio-lab-user/webui_xl.ipynb
